In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
from sklearn.preprocessing import scale 
from sklearn import metrics
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,mean_squared_error
from sklearn.metrics import roc_auc_score,roc_curve
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from sklearn.model_selection import RandomizedSearchCV

from statsmodels.stats.diagnostic import normal_ad
from scipy.stats import shapiro
from sklearn.preprocessing import StandardScaler


from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn import tree
from sklearn import svm
from sklearn.svm import SVC, SVR
from sklearn.cluster import KMeans

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier
#from xgboost import XGBClassifier
#from lightgbm import LGBMClassifier
#from catboost import CatBoostClassifier

from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import MinMaxScaler

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor 

from sklearn.tree import export_graphviz
from sklearn.externals.six import StringIO  
from IPython.display import Image  
import pydotplus
from termcolor import colored

from sklearn.decomposition import PCA

from warnings import filterwarnings
filterwarnings('ignore')

C:\Users\Asiye\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [2]:


class Information:
    
    def __init__(self,data):
        self.data = data;
        
    def data_features(self):
        print("--------------------------  DATA HEAD --------------------------\n")
        print(self.data.head())
        print()
        
        print("--------------------------  DATA DESCRIBE --------------------------\n")
        print(self.data.describe().T)
        print()

        print("--------------------------  DATA INFO --------------------------\n")
        print(self.data.info())
        print()
     
        print("--------------------------  DATA SHAPE --------------------------\n")
        print(self.data.shape)
        print()
    

In [3]:
class Visualizer:
    def __init__(self):
        print()
    
    def barplot(self,column1,column2):
        pd.crosstab(column1,column2).plot(kind='bar')
        plt.show()
    
    
    def missing_values(self,data):
        self.data = data
        plt.subplots(figsize=(12, 6))
        plt.title('Missing Values')
        sns.heatmap(self.data.isnull(), yticklabels = False, cmap="viridis")
        plt.show()
        
    def confusion_matris(self,y,y_pred):
        cm = confusion_matrix(y, y_pred)

        fig, ax = plt.subplots(figsize=(5, 5))
        ax.imshow(cm)
        ax.grid(False)
        ax.xaxis.set(ticks=(0, 1), ticklabels=('Predicted 0s', 'Predicted 1s'))
        ax.yaxis.set(ticks=(0, 1), ticklabels=('Actual 0s', 'Actual 1s'))
        ax.set_ylim(1.5, -0.5)
        for i in range(2):
            for j in range(2):
                ax.text(j, i, cm[i, j], ha='center', va='center', color='red')
        plt.show()
        
    def corrMatrix(self,data):
        corr = data.corr()
        plt.figure(figsize=(20,20))
        
        ax = sns.heatmap(
            corr, 
            vmin=-1, vmax=1, center=0,
            cmap=sns.diverging_palette(20, 220, n=200),
            square=True
        )
        ax.set_xticklabels(
            ax.get_xticklabels(),
            rotation=45,
            horizontalalignment='right'
        );
        
    def Elbow(self,range_nums,data):
        wcss = []
 
        for k in range_nums: 
            #Building and fitting the model 
            kmeanModel = KMeans(n_clusters=k)
            kmeanModel.fit(data)     


            wcss.append(kmeanModel.inertia_) 

        plt.plot(range_nums,wcss)
        plt.xlabel("Number of k (cluster) value")
        plt.ylabel("wcss")
        plt.show()
        
        
    def GraphTree(self,model,X_train):  # Çalışmıyooor!!
        
        feature_cols = X_train.columns
        
        dot_data = StringIO()
        export_graphviz(model, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True,feature_names = feature_cols,class_names=['0','1'])
        graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
        #graph.write_png('DecisionTreeCls.png')
        Image(graph.create_png())
        
    def ROCCurve(self,X_test,y_test,y_pred,model):
        
        logit_roc_auc = roc_auc_score(y_test,y_pred)

        fpr, tpr, thresholds = roc_curve(y_test, model.predict_proba(X_test)[:,1])

        plt.figure()
        plt.plot(fpr, tpr, label='AUC (area = %0.2f)' % logit_roc_auc)
        plt.plot([0, 1], [0, 1],'r--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Oranı')
        plt.ylabel('True Positive Oranı')
        plt.title('ROC')
        plt.show()
        
    # Linear Reg Assumptions

    def calculate_residuals(model, features, label):
        """
        Creates predictions on the features with the model and calculates residuals
        """
        predictions = model.predict(features)
        df_results = pd.DataFrame({'Actual': label, 'Predicted': predictions})
        df_results['Residuals'] = abs(df_results['Actual']) - abs(df_results['Predicted'])

        return df_results    
    
    def normal_errors_assumption(self,model, features, label, p_value_thresh=0.05):
              
        
        """
        Normality: Assumes that the error terms are normally distributed. If they are not,
        nonlinear transformations of variables may solve this.

        This assumption being violated primarily causes issues with the confidence intervals
        """
    
        print('Assumption 2: The error terms are normally distributed', '\n')
    
    # Calculating residuals for the Anderson-Darling test
        df_results = Visualizer.calculate_residuals(model, features, label)
    
        print('Using the Anderson-Darling test for normal distribution')

    # Performing the test on the residuals
        p_value = normal_ad(df_results['Residuals'])[1]
        print('p-value from the test - below 0.05 generally means non-normal:', p_value)
    
    # Reporting the normality of the residuals
        if p_value < p_value_thresh:
            print('Residuals are not normally distributed')
        else:
            print('Residuals are normally distributed')
    
    # Plotting the residuals distribution
        plt.subplots(figsize=(12, 6))
        plt.title('Distribution of Residuals')
        sns.distplot(df_results['Residuals'])
        plt.show()
    
        print()
        if p_value > p_value_thresh:
            print('Assumption satisfied')
        else:
            print('Assumption not satisfied')
            print()
            print('Confidence intervals will likely be affected')
            print('Try performing nonlinear transformations on variables')
    
    def linear_assumption(self,model, features, label):
        """
        Linearity: Assumes that there is a linear relationship between the predictors and
                   the response variable. If not, either a quadratic term or another
                   algorithm should be used.
        """
        print('Assumption 1: Linear Relationship between the Target and the Feature', '\n')

        print('Checking with a scatter plot of actual vs. predicted.',
               'Predictions should follow the diagonal line.')

        # Calculating residuals for the plot
        df_results = Visualizer.calculate_residuals(model, features, label)

        # Plotting the actual vs predicted values
        sns.lmplot(x='Actual', y='Predicted', data=df_results, fit_reg=False, size=7)

        # Plotting the diagonal line
        line_coords = np.arange(df_results.min().min(), df_results.max().max())
        plt.plot(line_coords, line_coords,  # X and y points
                 color='darkorange', linestyle='--')
        plt.title('Actual vs. Predicted')
        plt.show()
        
    def homoscedasticity_assumption(self,model, features, label):
        """
        Homoscedasticity: Assumes that the errors exhibit constant variance
        """
        print('Assumption 5: Homoscedasticity of Error Terms', '\n')

        print('Residuals should have relative constant variance')

        # Calculating residuals for the plot
        df_results = Visualizer.calculate_residuals(model, features, label)

        # Plotting the residuals
        plt.subplots(figsize=(12, 6))
        ax = plt.subplot(111)  # To remove spines
        plt.scatter(x=df_results.index, y=df_results.Residuals, alpha=0.5)
        plt.plot(np.repeat(0, df_results.index.max()), color='darkorange', linestyle='--')
        ax.spines['right'].set_visible(False)  # Removing the right spine
        ax.spines['top'].set_visible(False)  # Removing the top spine
        plt.title('Residuals')
        plt.show() 
        
        


    def autocorrelation_assumption(self,model, features, label):
        """
        Autocorrelation: Assumes that there is no autocorrelation in the residuals. If there is
                         autocorrelation, then there is a pattern that is not explained due to
                         the current value being dependent on the previous value.
                         This may be resolved by adding a lag variable of either the dependent
                         variable or some of the predictors.
        """
        from statsmodels.stats.stattools import durbin_watson
        print('Assumption 4: No Autocorrelation', '\n')

        # Calculating residuals for the Durbin Watson-tests
        df_results = Visualizer.calculate_residuals(model, features, label)

        print('\nPerforming Durbin-Watson Test')
        print('Values of 1.5 < d < 2.5 generally show that there is no autocorrelation in the data')
        print('0 to 2< is positive autocorrelation')
        print('>2 to 4 is negative autocorrelation')
        print('-------------------------------------')
        durbinWatson = durbin_watson(df_results['Residuals'])
        print('Durbin-Watson:', durbinWatson)
        if durbinWatson < 1.5:
            print('Signs of positive autocorrelation', '\n')
            print('Assumption not satisfied')
        elif durbinWatson > 2.5:
            print('Signs of negative autocorrelation', '\n')
            print('Assumption not satisfied')
        else:
            print('Little to no autocorrelation', '\n')
            print('Assumption satisfied')


In [4]:
# enum, kayıp gözlem var mı , varsa hangi yöntemlerle doldurulmalı
class Preprocess: 

    
    def __init__(self,data):
        self.data = data
        
    def get_missing_values(self):
        missing_values = self.data.isnull().sum()
        missing_values.sort_values(ascending=False, inplace=True)
        print(colored("------------  Missing Values  -------------",'blue'))
        return missing_values
    
    # column lardaki boş olan yerleri hangi metoda göre dolduracağımız: mean, medyan,mod ya da seçeceğimiz belirli bir değer
    
    def imputation(self,column,method):
        
        if method == "median":
            self.data[column] = self.data[column].fillna(self.data[column].median())
            
        elif method == "mode":
            self.data[column] = self.data[column].fillna(self.data[column].mode())
            
        elif method == "mean":
            self.data[column] = self.data[column].fillna(self.data[column].mean())
            
        else:
             self.data[column] = self.data[column].fillna(method)
                
        return self.get_missing_values()
    
    def drop(self,method):
        
        if method == "any":   #herhangi bir satırda NaN değeri varsa o satırı sil demek.
            print("Drop Öncesi Data Shape -->   ",self.data.shape)
            self.data.dropna(how="any",inplace=True)
            print("Drop Sonrası Data Shape -->   ",self.data.shape)
            
        elif method == "all":   #Tüm satır nan değerinde ise siler.
            self.data.dropna(how="all",inplace=True)
        
        # !!!!
        
        else:                  #sadece datanın method (method değerini data columnlarından birini yazarsak) oranındaki nan valuları siler.
            self.data.dropna(subset=[method],inplace=True)
            
        
        return self.get_missing_values()
    
    
    def SMOTE(self, X, y):
        
        os = SMOTE(random_state=0)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
        columns = X_train.columns
        os_data_X,os_data_y=os.fit_sample(X_train, y_train)
        os_data_X = pd.DataFrame(data=os_data_X,columns=columns )
        os_data_y= pd.DataFrame(data=os_data_y,columns=y_train.columns)
        
        return os_data_X, os_data_y
        
        
        
        


In [5]:
class GridSearchHelper:
    
    
    def __init__(self):
        print("Grid Search Helper...")
        index = ["Logistic", "Decision Tree","Random Forest","Neural Network","SVM", "BernoulliNB", "GaussianNB","MultinomialNB" , "KNN"]
        columns = ["Train","Test","Tuned"]
        self.Scores = pd.DataFrame(columns =columns, index = index)
        
        index_ = ["Linear", "Decision Tree","Random Forest","Neural Network","SVM","KNN"]
        columns_ = ["RMSE","RMSE_MinMaxScaled","RMSE_StandardScaled", "PCA","Tuned"]
        self.RMSE = pd.DataFrame(columns =columns_, index = index_)
        
        
        
    def AllScores(self):
        return self.Scores
    
    def AllRMSE(self):
        return self.RMSE
    
    
    
    def LogisticReg(self,X_train,X_test,y_train,y_test):
        
        print(colored('------------------     STATS  MODELS  --------------------\n','blue'))
        lj = sm.Logit(y_train,X_train)
        ljm = lj.fit()
        
        print(ljm.summary())
        
        print()
        print(colored('------------------     SCIKIT LEARN MODEL  --------------------\n','blue'))
        
        #Scikit Learn Model
        loj =LogisticRegression(solver = "liblinear")

        loj_model = loj.fit(X_train,y_train)
        train_accuracy = accuracy_score(y_train,loj_model.predict(X_train))
        test_accuracy = accuracy_score(y_test,loj_model.predict(X_test))

        print(colored("Intercept :  ",'red'),loj_model.intercept_)
        print()
        print(colored("Coefficient  : ",'red'),loj_model.coef_)
        print()
        
        print(colored('TRAIN','red'))
      
        visualize = Visualizer()
        
        visualize.confusion_matris(y_train,loj_model.predict(X_train))
        print(classification_report(y_train,loj_model.predict(X_train)))
        print(colored("Train accuracy score : ",'blue'),train_accuracy)
       
        
        print(colored('TEST','red'))
      
        visualize = Visualizer()
        visualize.confusion_matris(y_test, loj_model.predict(X_test))
        print(classification_report(y_test,loj_model.predict(X_test)))
        print(colored("Test accuracy score : ",'blue'), test_accuracy)
        
        self.Scores["Train"]["Logistic"] = train_accuracy
        self.Scores["Test"]["Logistic"] = test_accuracy
    
   
        return loj_model  , loj_model.predict(X_test)
    
        
           
    
    def DecisionTreeCls(self,X_train,X_test,y_train,y_test):
           
        # Model
        
        clf = DecisionTreeClassifier()
        clf = clf.fit(X_train,y_train)
        train_accuracy = accuracy_score(y_train,clf.predict(X_train))
        test_accuracy = accuracy_score(y_test,clf.predict(X_test))
      
        print(colored('TRAIN','red'))
      
        visualize = Visualizer()
        
        visualize.confusion_matris(y_train, clf.predict(X_train))
        print(classification_report(y_train,clf.predict(X_train)))
        print(colored("Train accuracy score : ",'blue'),train_accuracy)
       
        
        print(colored('TEST','red'))
      
       
        visualize.confusion_matris(y_test, clf.predict(X_test))
        print(classification_report(y_test,clf.predict(X_test)))
        print(colored("Test accuracy score : ",'blue'), test_accuracy)
        
        self.Scores["Train"]["Decision Tree"] = train_accuracy
        self.Scores["Test"]["Decision Tree"] = test_accuracy
    
   
        return clf


    def RandomForestCls(self,X_train,X_test,y_train,y_test):
           
        # Model
        
        clf = RandomForestClassifier()
        clf = clf.fit(X_train,y_train)
        train_accuracy = accuracy_score(y_train,clf.predict(X_train))
        test_accuracy = accuracy_score(y_test,clf.predict(X_test))
      
        print(colored('TRAIN','red'))
      
        visualize = Visualizer()
        visualize.confusion_matris(y_train, clf.predict(X_train))
        print(classification_report(y_train,clf.predict(X_train)))
        print(colored("Train accuracy score : ",'blue'),train_accuracy)
       
        
        print(colored('TEST','red'))
      
        
        visualize.confusion_matris(y_test, clf.predict(X_test))
        print(classification_report(y_test,clf.predict(X_test)))
        print(colored("Test accuracy score : ",'blue'), test_accuracy)
        
        self.Scores["Train"]["Random Forest"] = train_accuracy
        self.Scores["Test"]["Random Forest"] = test_accuracy
    
   
        return clf

    def NeuralNetworkCls(self,X_train,X_test,y_train,y_test):
           
        # Model
        
        clf = MLPClassifier()
        clf = clf.fit(X_train,y_train)
        train_accuracy = accuracy_score(y_train,clf.predict(X_train))
        test_accuracy = accuracy_score(y_test,clf.predict(X_test))
      
        print(colored('TRAIN','red'))
      
        visualize = Visualizer()
        
        visualize.confusion_matris(y_train, clf.predict(X_train))
        print(classification_report(y_train,clf.predict(X_train)))
        print(colored("Train accuracy score : ",'blue'),train_accuracy)
       
        
        print(colored('TEST','red'))
      
        
        visualize.confusion_matris(y_test, clf.predict(X_test))
        print(classification_report(y_test,clf.predict(X_test)))
        print(colored("Test accuracy score : ",'blue'), test_accuracy)
        
        self.Scores["Train"]["Neural Network"] = train_accuracy
        self.Scores["Test"]["Neural Network"] = test_accuracy
    
   
        return clf

    def GaussianNBCls(self,X_train,X_test,y_train,y_test):
           
        # Model
        
        clf = GaussianNB()
        clf = clf.fit(X_train,y_train)
        train_accuracy = accuracy_score(y_train,clf.predict(X_train))
        test_accuracy = accuracy_score(y_test,clf.predict(X_test))
      
        print(colored('TRAIN','red'))
      
        visualize = Visualizer()
        
        visualize.confusion_matris(y_train, clf.predict(X_train))
        print(classification_report(y_train,clf.predict(X_train)))
        print(colored("Train accuracy score : ",'blue'),train_accuracy)
       
        
        print(colored('TEST','red'))
      
       
        visualize.confusion_matris(y_test, clf.predict(X_test))
        print(classification_report(y_test,clf.predict(X_test)))
        print(colored("Test accuracy score : ",'blue'), test_accuracy)
        
        self.Scores["Train"]["GaussianNB"] = train_accuracy
        self.Scores["Test"]["GaussianNB"] = test_accuracy
    
   
        return clf

    def BernoulliNBCls(self,X_train,X_test,y_train,y_test):
           
        # Model
        
        clf = BernoulliNB()
        clf = clf.fit(X_train,y_train)
        train_accuracy = accuracy_score(y_train,clf.predict(X_train))
        test_accuracy = accuracy_score(y_test,clf.predict(X_test))
      
        print(colored('TRAIN','red'))
      
        visualize = Visualizer()
        
        visualize.confusion_matris(y_train, clf.predict(X_train))
        print(classification_report(y_train,clf.predict(X_train)))
        print(colored("Train accuracy score : ",'blue'),train_accuracy)
       
        
        print(colored('TEST','red'))
      
        
        visualize.confusion_matris(y_test, clf.predict(X_test))
        print(classification_report(y_test,clf.predict(X_test)))
        print(colored("Test accuracy score : ",'blue'), test_accuracy)
        
        self.Scores["Train"]["BernoulliNB"] = train_accuracy
        self.Scores["Test"]["BernoulliNB"] = test_accuracy
    
   
        return clf

    def MultinomialNBCls(self,X_train,X_test,y_train,y_test):
           
        # Model
        
        clf = MultinomialNB()
        clf = clf.fit(X_train,y_train)
        train_accuracy = accuracy_score(y_train,clf.predict(X_train))
        test_accuracy = accuracy_score(y_test,clf.predict(X_test))
      
        print(colored('TRAIN','red'))
      
        visualize = Visualizer()
        
        visualize.confusion_matris(y_train, clf.predict(X_train))
        print(classification_report(y_train,clf.predict(X_train)))
        print(colored("Train accuracy score : ",'blue'),train_accuracy)
       
        
        print(colored('TEST','red'))
      
        
        visualize.confusion_matris(y_test, clf.predict(X_test))
        print(classification_report(y_test,clf.predict(X_test)))
        print(colored("Test accuracy score : ",'blue'), test_accuracy)
        
        self.Scores["Train"]["MultinomialNB"] = train_accuracy
        self.Scores["Test"]["MultinomialNB"] = test_accuracy
    
   
        return clf
 

    def SVMCls(self,X_train,X_test,y_train,y_test):
           
        # Model
        
        clf = SVC()
        clf = clf.fit(X_train,y_train)
        train_accuracy = accuracy_score(y_train,clf.predict(X_train))
        test_accuracy = accuracy_score(y_test,clf.predict(X_test))
      
        print(colored('TRAIN','red'))
      
        visualize = Visualizer()
        
        visualize.confusion_matris(y_train, clf.predict(X_train))
        print(classification_report(y_train,clf.predict(X_train)))
        print(colored("Train accuracy score : ",'blue'),train_accuracy)
       
        
        print(colored('TEST','red'))
      
        
        visualize.confusion_matris(y_test, clf.predict(X_test))
        print(classification_report(y_test,clf.predict(X_test)))
        print(colored("Test accuracy score : ",'blue'), test_accuracy)
        
        self.Scores["Train"]["SVM"] = train_accuracy
        self.Scores["Test"]["SVM"] = test_accuracy
    
   
        return clf

    def KNNCls(self,X_train,X_test,y_train,y_test):
           
        # Model
        
        clf =  KNeighborsClassifier()
        clf = clf.fit(X_train,y_train)
        train_accuracy = accuracy_score(y_train,clf.predict(X_train))
        test_accuracy = accuracy_score(y_test,clf.predict(X_test))
      
        print(colored('TRAIN','red'))
      
        visualize = Visualizer()
        
        visualize.confusion_matris(y_train, clf.predict(X_train))
        print(classification_report(y_train,clf.predict(X_train)))
        print(colored("Train accuracy score : ",'blue'),train_accuracy)
       
        
        print(colored('TEST','red'))
      
        
        visualize.confusion_matris(y_test, clf.predict(X_test))
        print(classification_report(y_test,clf.predict(X_test)))
        print(colored("Test accuracy score : ",'blue'), test_accuracy)
        
        self.Scores["Train"]["KNN"] = train_accuracy
        self.Scores["Test"]["KNN"] = test_accuracy
    
   
        return clf

# ----------------- Regression ---------------------------

    def LinearReg(self,X_train,X_test,y_train,y_test):
        
        reg = LinearRegression()
        reg.fit(X_train,y_train)
        y_pred = reg.predict(X_test)
        
        print("RMSE : ", np.sqrt(mean_squared_error(y_test, y_pred)))
        
        
        self.RMSE["RMSE"]["Linear"] = np.sqrt(mean_squared_error(y_test, y_pred))
        return reg
        


    def DecisionTreeReg(self,X_train,X_test,y_train,y_test):
        
        reg = DecisionTreeRegressor()
        reg.fit(X_train,y_train)
        y_pred = reg.predict(X_test)
        
        print("RMSE : ", np.sqrt(mean_squared_error(y_test, y_pred)))
        
        self.RMSE["RMSE"]["Decision Tree"] = np.sqrt(mean_squared_error(y_test, y_pred))
        
        return reg

    def RandomForestReg(self,X_train,X_test,y_train,y_test):
        
        reg = RandomForestRegressor()
        reg.fit(X_train,y_train)
        y_pred = reg.predict(X_test)
        
        print("RMSE : ", np.sqrt(mean_squared_error(y_test, y_pred)))
        
        self.RMSE["RMSE"]["Random Forest"] = np.sqrt(mean_squared_error(y_test, y_pred))
        return reg
 
    def NeuralNetworkReg(self,X_train,X_test,y_train,y_test):
        
        reg = MLPRegressor()
        reg.fit(X_train,y_train)
        y_pred = reg.predict(X_test)
        
        print("RMSE : ", np.sqrt(mean_squared_error(y_test, y_pred)))
        
        self.RMSE["RMSE"]["Neural Network"] = np.sqrt(mean_squared_error(y_test, y_pred))
        return reg
    
    def SVMReg(self,X_train,X_test,y_train,y_test):
        
        reg = SVR()
        reg.fit(X_train,y_train)
        y_pred = reg.predict(X_test)
        
        print("RMSE : ", np.sqrt(mean_squared_error(y_test, y_pred)))
        
        self.RMSE["RMSE"]["SVM"] = np.sqrt(mean_squared_error(y_test, y_pred))
        return reg

    def KNNReg(self,X_train,X_test,y_train,y_test):
        
        reg = KNeighborsRegressor()
        reg.fit(X_train,y_train)
        y_pred = reg.predict(X_test)
        
        print("RMSE : ", np.sqrt(mean_squared_error(y_test, y_pred)))
        
        self.RMSE["RMSE"]["KNN"] = np.sqrt(mean_squared_error(y_test, y_pred))
        return reg
        
    


    def GridSearchCVM(self,model_name,model,params,X_train,X_test,y_train,y_test):
        grid = GridSearchCV(model, params, cv =10)
        grid.fit(X_train, y_train)
        print("En iyi parametresi :"+ str(grid.best_params_))
        best_params = grid.best_estimator_
        predict = best_params.predict(X_test)
        accuracy_tuned = accuracy_score(y_test,predict)
        print("Tuned Model Accuracy Score : ",accuracy_tuned)
        self.Scores["Tuned"][model_name] = accuracy_tuned
        
        return best_params
        
    def RandomizedSearchCVM(self, model_name, model,params,X_train,X_test,y_train,y_test):
        rv = RandomizedSearchCV(model,params,n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
        rv.fit(X_train, y_train)
        print("En iyi parametresi :"+ str(rv.best_params_))
        best_params = rv.best_estimator_
        predict = best_params.predict(X_test)
        accuracy_tuned = accuracy_score(y_test,predict)
        print("Tuned Model Accuracy Score : ",accuracy_tuned)
        self.Scores["Tuned"][model_name] = accuracy_tuned
        
        return best_params
        

    def MinMaxScaleModel(self,model_name,model,X_train,X_test,y_train,y_test):
        
        
        scaler = MinMaxScaler()
        scaler.fit(X_train)
        X_train_s = scaler.transform(X_train)
        X_test_s = scaler.transform(X_test)
        
        model.fit(X_train_s,y_train)
        y_pred = model.predict(X_test_s)
        
        print("RMSE : ", np.sqrt(mean_squared_error(y_test, y_pred)))
        
        self.RMSE["RMSE_MinMaxScaled"][model_name] = np.sqrt(mean_squared_error(y_test, y_pred))
        
        return X_train_s, X_test_s
        
    
    def StandardScaleModel(self,model_name,model,X_train,X_test,y_train,y_test):
        
        
        scaler = StandardScaler()
        scaler.fit(X_train)
        X_train_st = scaler.transform(X_train)
        X_test_st = scaler.transform(X_test)
        
        model.fit(X_train_st,y_train)
        y_pred = model.predict(X_test_st)
        
        print("RMSE : ", np.sqrt(mean_squared_error(y_test, y_pred)))
        
        self.RMSE["RMSE_StandardScaled"][model_name] = np.sqrt(mean_squared_error(y_test, y_pred))
        
        
        return X_train_st, X_test_st
        

    def GridSearchCVR(self,model_name,model,params,X_train,X_test,y_train,y_test):
        grid = GridSearchCV(model, params, cv =10)
        grid.fit(X_train, y_train)
        print("En iyi parametresi :"+ str(grid.best_params_))
        best_params = grid.best_estimator_
        predict = best_params.predict(X_test)
        rmse_tuned = np.sqrt(mean_squared_error(y_test, predict))
        print("Tuned Model RMSE : ",rmse_tuned)
        self.RMSE["Tuned"][model_name] = rmse_tuned
        
        return best_params
        
    def RandomizedSearchCVR(self, model_name, model,params,X_train,X_test,y_train,y_test):
        rv = RandomizedSearchCV(model,params,n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
        rv.fit(X_train, y_train)
        print("En iyi parametresi :"+ str(rv.best_params_))
        best_params = rv.best_estimator_
        predict = best_params.predict(X_test)
        rmse_tuned = np.sqrt(mean_squared_error(y_test, predict))
        print("Tuned Model RMSE : ",rmse_tuned)
        self.RMSE["Tuned"][model_name] = rmse_tuned       
        
        return best_params
    

    def PCAModel(self,model_name,model,X_train,X_test,y_train,y_test):
        pca = PCA()
        X_train_pca = pca.fit_transform(X_train)
        X_test_pca = pca.transform(X_test)
        model.fit(X_train_pca,y_train)
        y_pred = model.predict(X_test_pca)
        
        self.RMSE["PCA"][model_name] = np.sqrt(mean_squared_error(y_test, y_pred))
        
    
    def pca_(self,X):
        pca = PCA()
        X_pca = pca.fit_transform(X)
        return X_pca
    
    def minmaxscale(self,X):
        scaler = MinMaxScaler()
        scaler.fit(X)
        X_s = scaler.transform(X)
        return X
    
    
    def Kmeans(self,data,n_cluster):
        kmeans = KMeans(n_clusters = n_cluster)

        clusters = kmeans.fit_predict(data)
        return clusters
    